In [2]:
import pandas as pd
import glob

In [3]:
import seaborn as sns
import matplotlib.pyplot as plt

In [4]:
%pip install --quiet optuna

Note: you may need to restart the kernel to use updated packages.


In [5]:
import optuna
import os

/home/duarbrun/.local/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [6]:
optuna.__version__

'4.2.1'

In [ ]:
os.chdir("drive/MyDrive/Mestrado/GraphColor/results_abc_v1")

In [6]:
!ls

 AnalyseParameters.ipynb	       main_test_tabu_search
 Optuna.ipynb			       main_test_tabu_search.cpp
 Optuna.ipynb:Zone.Identifier	       resultado_calibrar_parametro
 README.md			       results_abc
 documentacao.md		       results_abc_2
 include			       results_abc_3
 instance_calibrar.csv		       results_abc_tabu
 instances			       results_abc_v2
 instances_info.csv		       results_grasp
'instances_with_k copy.csv'	       results_greedy
 instances_with_k.csv		       results_sa
 instances_with_k2.csv		       results_tabu
 instances_with_k_abc_2.csv	       run_abc
 instances_with_k_analysis.csv	       run_abc_calib
 instances_with_k_no_greedy_zero.csv   run_grasp
 lib				       run_greedy
 main				       run_main_abc.cpp
 main.cpp			       run_main_abc_calibrar_parametros.cpp
 main_test_abc_v2		       run_main_abc_until_zero.cpp
 main_test_abc_v2.cpp		       run_main_abc_v2
 main_test_fwlist		       run_main_abc_v2.cpp
 main_test_fwlist.cpp		       run_main_grasp.cpp
 main_test_gra

## ABC V1

In [ ]:
# Definir o caminho da pasta onde estão os arquivos TXT
folder_path = "results_abc/random/"  # Substitua pelo caminho correto

# Lista todos os arquivos TXT na pasta
txt_files = glob.glob(folder_path + "*.txt")

# Definir os nomes das colunas (ajuste conforme os dados)
columns = ["fitness", "execution_time", "num_bees", "limit", "max_iter",
           "filename", "num_vertices", "num_edges"]

txt_files

In [ ]:
# Lista para armazenar os DataFrames individuais
dfs = []

# Ler cada arquivo TXT e adicionar ao DataFrame
for file in txt_files:
    df = pd.read_csv(file, sep=";", names=columns, header=None)
    df['graph'] = file.split('/')[2].replace('.txt', '')
    dfs.append(df)

# Concatenar todos os DataFrames em um só
df_final = pd.concat(dfs, ignore_index=True)

# Exibir as primeiras linhas do DataFrame consolidado
df_final.head()

In [ ]:
df_final[df_final['graph']==None]

In [ ]:
# Normalizar a fitness dentro de cada grafo (opcional, mas pode ajudar)
df_final["fitness_norm"] = df_final.groupby("graph")["fitness"].transform(lambda x: (x - x.min()) / (x.max() - x.min()) if x.max() != x.min() else 0)

df_final

In [ ]:
def objective(trial):
    """Função objetivo para o Optuna, buscando a melhor configuração de parâmetros já testada."""

    # Optuna irá escolher um conjunto de parâmetros dentro dos valores existentes
    num_bees = trial.suggest_categorical("num_bees", df_final["num_bees"].unique().tolist())
    limit = trial.suggest_categorical("limit", df_final["limit"].unique().tolist())
    max_iter = trial.suggest_categorical("max_iter", df_final["max_iter"].unique().tolist())

    # Filtrar apenas as execuções com esses parâmetros
    subset = df_final[(df_final["num_bees"] == num_bees) &
                (df_final["limit"] == limit) &
                (df_final["max_iter"] == max_iter)]

    # Média das fitness normalizadas como critério de avaliação
    fitness_score = subset["fitness_norm"].mean()

    return fitness_score  # O Optuna busca minimizar esse valor

In [ ]:
# Criar e rodar a otimização
study = optuna.create_study(direction="minimize")
study.optimize(objective, n_trials=100)  # Ajuste o número de trials conforme necessário

# Exibir os melhores parâmetros encontrados
print("Melhores parâmetros encontrados:")
print(study.best_params)

## ABC V2

### Calibrar parâmetros

In [6]:
!ls

 Optuna.ipynb			       main_test_greedy.cpp
 Optuna.ipynb:Zone.Identifier	       main_test_methods.cpp
 README.md			       main_test_sa
 documentacao.md		       main_test_simulated_annealing.cpp
 include			       main_test_tabu_search
 instance_calibrar.csv		       main_test_tabu_search.cpp
 instances			       resultado_calibrar_parametro
 instances_info.csv		       results_abc
'instances_with_k copy.csv'	       results_abc_2
 instances_with_k.csv		       results_grasp
 instances_with_k2.csv		       results_greedy
 instances_with_k_abc_2.csv	       run_abc
 instances_with_k_no_greedy_zero.csv   run_abc_calib
 lib				       run_grasp
 main				       run_greedy
 main.cpp			       run_main_abc.cpp
 main_test_fwlist		       run_main_abc_calibrar_parametros.cpp
 main_test_fwlist.cpp		       run_main_abc_until_zero.cpp
 main_test_grasp		       run_main_grasp.cpp
 main_test_grasp.cpp		       run_main_greedy.cpp
 main_test_greedy		       shell_scripts


In [7]:
columns = ['conflitos', 'tempo_us', 'iter_found_best', 'num_bees', 
           'limit', 'max_iter', 'filename', 'rcl_size', 'random_search',
           'num_vertices', 'num_edges' ]
df = pd.read_csv('resultado_calibrar_parametro/DSJC500.5.txt', sep=';', names=columns, header=None)
df.head()

,conflitos,tempo_us,iter_found_best,num_bees,limit,max_iter,filename,rcl_size,random_search,num_vertices,num_edges
0,165,899234,0,50,5,20,instances/DSJC500.5.col,1,0,500,62624
1,165,875770,0,50,5,20,instances/DSJC500.5.col,1,0,500,62624
2,169,890426,0,50,5,20,instances/DSJC500.5.col,1,0,500,62624
3,158,878147,0,50,5,20,instances/DSJC500.5.col,1,0,500,62624
4,167,883116,0,50,5,20,instances/DSJC500.5.col,1,0,500,62624


In [8]:
# df['fitness_norm'] = df["fitness"].transform(lambda x: (x - x.min()) / (x.max() - x.min()) if x.max() != x.min() else 0)

In [9]:
def objective_v2(trial):
    """Função objetivo para o Optuna, buscando a melhor configuração de parâmetros já testada."""

    # Optuna irá escolher um conjunto de parâmetros dentro dos valores existentes
    num_bees = trial.suggest_categorical("num_bees", df["num_bees"].unique().tolist())
    limit = trial.suggest_categorical("limit", df["limit"].unique().tolist())
    max_iter = trial.suggest_categorical("max_iter", df["max_iter"].unique().tolist())
    random_search = trial.suggest_categorical("random_search", df["random_search"].unique().tolist())

    # Filtrar apenas as execuções com esses parâmetros
    subset = df[(df["num_bees"] == num_bees) &
                (df["limit"] == limit) &
                (df["max_iter"] == max_iter) &
                (df["random_search"] == random_search)]

    # Média das fitness normalizadas como critério de avaliação
    fitness_score = subset["conflitos"].mean()

    return fitness_score  # O Optuna busca minimizar esse valor

In [12]:
n_trials = df[["num_bees", "limit", "max_iter", "random_search"]].drop_duplicates().shape[0]

# Criar e rodar a otimização
study = optuna.create_study(direction="minimize")
study.optimize(objective_v2, n_trials=n_trials)  # Ajuste o número de trials conforme necessário

# Exibir os melhores parâmetros encontrados
print("Melhores parâmetros encontrados:")
print(study.best_params)

[I 2025-03-05 16:44:38,643] A new study created in memory with name: no-name-461f49fe-e9fa-4f42-aee6-8f77839823ed
[I 2025-03-05 16:44:38,653] Trial 0 finished with value: 165.4 and parameters: {'num_bees': 100, 'limit': 10, 'max_iter': 50, 'random_search': 0}. Best is trial 0 with value: 165.4.
[I 2025-03-05 16:44:38,656] Trial 1 finished with value: 162.1 and parameters: {'num_bees': 100, 'limit': 25, 'max_iter': 80, 'random_search': 0}. Best is trial 1 with value: 162.1.
[I 2025-03-05 16:44:38,660] Trial 2 finished with value: 159.7 and parameters: {'num_bees': 300, 'limit': 10, 'max_iter': 50, 'random_search': 0}. Best is trial 2 with value: 159.7.
[I 2025-03-05 16:44:38,664] Trial 3 finished with value: 161.4 and parameters: {'num_bees': 400, 'limit': 5, 'max_iter': 80, 'random_search': 1}. Best is trial 2 with value: 159.7.
[I 2025-03-05 16:44:38,667] Trial 4 finished with value: 165.3 and parameters: {'num_bees': 50, 'limit': 5, 'max_iter': 50, 'random_search': 1}. Best is trial 

Melhores parâmetros encontrados:
{'num_bees': 300, 'limit': 50, 'max_iter': 80, 'random_search': 1}


## ABCGraphColoring V2 - corrigido o uso da busca local e da seleção no onlooker

In [7]:
columns = ['conflitos', 'tempo_us', 'iter_found_best', 'num_bees', 
           'limit', 'max_iter', 'filename', 'method', 'metha',
           'num_vertices', 'num_edges', 'onlooker_phase' ]
df = pd.read_csv('results_abc_v2/calibrar_pseudo_greedy_v2/DSJC500.5.txt', sep=';', names=columns, header=None)
df.head()
df['metha'].unique()

array(['n'], dtype=object)

In [21]:
df['conflitos_norm'] = (df["conflitos"] - df["conflitos"].min()) / (df["conflitos"].max() - df["conflitos"].min() if df["conflitos"].max()!= df["conflitos"].min() else 0)
df['tempo_norm'] = (df["tempo_us"] - df["tempo_us"].min()) / (df["tempo_us"].max() - df["tempo_us"].min() if df["tempo_us"].max()!= df["tempo_us"].min() else 0)
def objective_v3(trial):
    """Função objetivo para o Optuna, buscando a melhor configuração de parâmetros já testada."""

    # Optuna irá escolher um conjunto de parâmetros dentro dos valores existentes
    num_bees = trial.suggest_categorical("num_bees", df["num_bees"].unique().tolist())
    limit = trial.suggest_categorical("limit", df["limit"].unique().tolist())
    max_iter = trial.suggest_categorical("max_iter", df["max_iter"].unique().tolist())
    onlooker_phase = trial.suggest_categorical("onlooker_phase", df["onlooker_phase"].unique().tolist())
    

    # Filtrar apenas as execuções com esses parâmetros
    subset = df[(df["num_bees"] == num_bees) &
                (df["limit"] == limit) &
                (df["max_iter"] == max_iter) &
                (df["onlooker_phase"] == onlooker_phase)]

    alpha = 1  # Peso para conflitos
    beta = 0   # Peso para tempo
    score = alpha * subset["conflitos_norm"].mean() + beta * subset["tempo_norm"].mean()
    

    return score  # O Optuna busca minimizar esse valor

In [33]:
n_trials = df[["num_bees", "limit", "max_iter", "onlooker_phase"]].drop_duplicates().shape[0]

# Criar e rodar a otimização
study = optuna.create_study(direction="minimize")
study.optimize(objective_v3, n_trials=n_trials)  # Ajuste o número de trials conforme necessário

# Exibir os melhores parâmetros encontrados
print("Melhores parâmetros encontrados:")
print(study.best_params)

[I 2025-03-30 13:01:38,620] A new study created in memory with name: no-name-15e0d24b-15fe-4f72-9c54-de1195e0914a
[I 2025-03-30 13:01:38,624] Trial 0 finished with value: 0.48125 and parameters: {'num_bees': 80, 'limit': 500, 'max_iter': 100, 'onlooker_phase': 'random_search_onlooker'}. Best is trial 0 with value: 0.48125.
[I 2025-03-30 13:01:38,627] Trial 1 finished with value: 0.515625 and parameters: {'num_bees': 100, 'limit': 250, 'max_iter': 100, 'onlooker_phase': 'random_search_onlooker'}. Best is trial 0 with value: 0.48125.
[I 2025-03-30 13:01:38,629] Trial 2 finished with value: 0.528125 and parameters: {'num_bees': 60, 'limit': 250, 'max_iter': 100, 'onlooker_phase': 'random_search_onlooker'}. Best is trial 0 with value: 0.48125.
[I 2025-03-30 13:01:38,632] Trial 3 finished with value: 0.7125 and parameters: {'num_bees': 70, 'limit': 10, 'max_iter': 1000, 'onlooker_phase': 'waggle_dance_onlooker'}. Best is trial 0 with value: 0.48125.
[I 2025-03-30 13:01:38,635] Trial 4 finis

Melhores parâmetros encontrados:
{'num_bees': 90, 'limit': 500, 'max_iter': 1000, 'onlooker_phase': 'waggle_dance_onlooker'}


In [27]:
df.conflitos.mean()

np.float64(168.09333333333333)

In [28]:
df.conflitos.std()

np.float64(5.05776034148998)

In [29]:
df.conflitos.min()

np.int64(150)

In [30]:
df.conflitos.max()

np.int64(182)

In [31]:
df.conflitos.describe()

count    1200.000000
mean      168.093333
std         5.057760
min       150.000000
25%       165.000000
50%       168.000000
75%       172.000000
max       182.000000
Name: conflitos, dtype: float64

## Versão após corrigir o random seed

In [23]:
columns_v4 = ['conflitos', 'tempo_us', 'iter_found_best', 'num_bees', 
           'limit', 'max_iter', 'filename', 'random_search',
           'num_vertices', 'num_edges' ]
df_v4 = pd.read_csv('results_abc_v3/calibrar/DSJC250.1.txt', sep=';', names=columns_v4, header=None)
df_v4.head()

,conflitos,tempo_us,iter_found_best,num_bees,limit,max_iter,filename,random_search,num_vertices,num_edges
0,75,749849,6,50,10,100,instances/DSJC250.1.col,p,250,3218
1,74,714548,1,50,10,100,instances/DSJC250.1.col,p,250,3218
2,60,1943199,177,50,10,100,instances/DSJC250.1.col,p,250,3218
3,71,812902,8,50,10,100,instances/DSJC250.1.col,p,250,3218
4,72,907787,25,50,10,100,instances/DSJC250.1.col,p,250,3218


In [24]:
df_v4['conflitos_norm'] = (df_v4["conflitos"] - df_v4["conflitos"].min()) / (df_v4["conflitos"].max() - df_v4["conflitos"].min() if df_v4["conflitos"].max()!= df_v4["conflitos"].min() else 0)
df_v4['tempo_norm'] = (df_v4["tempo_us"] - df_v4["tempo_us"].min()) / (df_v4["tempo_us"].max() - df_v4["tempo_us"].min() if df_v4["tempo_us"].max()!= df_v4["tempo_us"].min() else 0)
def objective_v4(trial):
    """Função objetivo para o Optuna, buscando a melhor configuração de parâmetros já testada."""

    # Optuna irá escolher um conjunto de parâmetros dentro dos valores existentes
    num_bees = trial.suggest_categorical("num_bees", df_v4["num_bees"].unique().tolist())
    limit = trial.suggest_categorical("limit", df_v4["limit"].unique().tolist())
    max_iter = trial.suggest_categorical("max_iter", df_v4["max_iter"].unique().tolist())
    

    # Filtrar apenas as execuções com esses parâmetros
    subset = df_v4[(df_v4["num_bees"] == num_bees) &
                (df_v4["limit"] == limit) &
                (df_v4["max_iter"] == max_iter)]

    alpha = 1  # Peso para conflitos
    beta = 0   # Peso para tempo
    score = alpha * subset["conflitos_norm"].mean() + beta * subset["tempo_norm"].mean()
    

    return score  # O Optuna busca minimizar esse valor

In [25]:
n_trials = df_v4[["num_bees", "limit", "max_iter"]].drop_duplicates().shape[0]

# Criar e rodar a otimização
study = optuna.create_study(direction="minimize")
study.optimize(objective_v4, n_trials=n_trials)  # Ajuste o número de trials conforme necessário

# Exibir os melhores parâmetros encontrados
print("Melhores parâmetros encontrados:")
print(study.best_params)

[I 2025-04-03 08:25:41,151] A new study created in memory with name: no-name-055533d7-b4b8-487c-a521-1501a7ccb245
[I 2025-04-03 08:25:41,165] Trial 0 finished with value: 0.5566666666666666 and parameters: {'num_bees': 80, 'limit': 250, 'max_iter': 10000}. Best is trial 0 with value: 0.5566666666666666.
[I 2025-04-03 08:25:41,167] Trial 1 finished with value: 0.7566666666666666 and parameters: {'num_bees': 90, 'limit': 250, 'max_iter': 100}. Best is trial 0 with value: 0.5566666666666666.
[I 2025-04-03 08:25:41,169] Trial 2 finished with value: 0.8316666666666667 and parameters: {'num_bees': 60, 'limit': 100, 'max_iter': 10}. Best is trial 0 with value: 0.5566666666666666.
[I 2025-04-03 08:25:41,170] Trial 3 finished with value: 0.6566666666666667 and parameters: {'num_bees': 50, 'limit': 50, 'max_iter': 1000}. Best is trial 0 with value: 0.5566666666666666.
[I 2025-04-03 08:25:41,172] Trial 4 finished with value: 0.23166666666666663 and parameters: {'num_bees': 90, 'limit': 5, 'max_it

[I 2025-04-03 08:25:41,177] Trial 7 finished with value: 0.8200000000000001 and parameters: {'num_bees': 50, 'limit': 10, 'max_iter': 10}. Best is trial 4 with value: 0.23166666666666663.
[I 2025-04-03 08:25:41,181] Trial 8 finished with value: 0.7883333333333333 and parameters: {'num_bees': 90, 'limit': 500, 'max_iter': 1000}. Best is trial 4 with value: 0.23166666666666663.
[I 2025-04-03 08:25:41,184] Trial 9 finished with value: 0.7883333333333333 and parameters: {'num_bees': 90, 'limit': 500, 'max_iter': 1000}. Best is trial 4 with value: 0.23166666666666663.
[I 2025-04-03 08:25:41,189] Trial 10 finished with value: 0.26499999999999996 and parameters: {'num_bees': 60, 'limit': 5, 'max_iter': 10000}. Best is trial 4 with value: 0.23166666666666663.
[I 2025-04-03 08:25:41,191] Trial 11 finished with value: 0.26499999999999996 and parameters: {'num_bees': 60, 'limit': 5, 'max_iter': 10000}. Best is trial 4 with value: 0.23166666666666663.
[I 2025-04-03 08:25:41,194] Trial 12 finished 

Melhores parâmetros encontrados:
{'num_bees': 80, 'limit': 5, 'max_iter': 10000}


## TABU SEARCH

In [6]:
files = ['4-FullIns_4', 'ash331GPIA', 'DSJC250.1', 'le450_15a', 'qg.order30', 'queen13_13']
columns_tabu = ['conflitos', 'tempo_us', 'iter_found_best', 
           'max_iter', 'T_iter', 'num_color', 'filename', 
           'num_vertices', 'num_edges', 'init_method' ]

In [11]:
dfs = []

for file in files:
    df = pd.read_csv(f'results_tabu_v2/analysis/{file}.txt', sep=';', names=columns_tabu, header=None)
    df['conflitos_norm'] = (df["conflitos"] - df["conflitos"].min()) / (df["conflitos"].max() - df["conflitos"].min() if df["conflitos"].max()!= df["conflitos"].min() else 0)
    df['tempo_norm'] = (df["tempo_us"] - df["tempo_us"].min()) / (df["tempo_us"].max() - df["tempo_us"].min() if df["tempo_us"].max()!= df["tempo_us"].min() else 0)
    dfs.append(df)

df_tabu = pd.concat(dfs, ignore_index=True)
df_tabu.tail()

,conflitos,tempo_us,iter_found_best,max_iter,T_iter,num_color,filename,num_vertices,num_edges,init_method,conflitos_norm,tempo_norm
5755,60,11048850,163,10000,100,13,instances/queen13_13.col,169,6656,random,0.21875,0.755890
5756,62,11650160,262,10000,100,13,instances/queen13_13.col,169,6656,random,0.25000,0.797101
5757,64,14143315,188,10000,100,13,instances/queen13_13.col,169,6656,random,0.28125,0.967969
5758,56,11127899,177,10000,100,13,instances/queen13_13.col,169,6656,random,0.15625,0.761307
5759,60,12387752,149,10000,100,13,instances/queen13_13.col,169,6656,random,0.21875,0.847651


In [14]:
df_tabu['conflitos_norm'] = (df_tabu["conflitos"] - df_tabu["conflitos"].min()) / (df_tabu["conflitos"].max() - df_tabu["conflitos"].min() if df_tabu["conflitos"].max()!= df_tabu["conflitos"].min() else 0)
df_tabu['tempo_norm'] = (df_tabu["tempo_us"] - df_tabu["tempo_us"].min()) / (df_tabu["tempo_us"].max() - df_tabu["tempo_us"].min() if df_tabu["tempo_us"].max()!= df_tabu["tempo_us"].min() else 0)
def objective_v4(trial):
    """Função objetivo para o Optuna, buscando a melhor configuração de parâmetros já testada."""

    # Optuna irá escolher um conjunto de parâmetros dentro dos valores existentes
    T_iter = trial.suggest_categorical("T_iter", df_tabu["T_iter"].unique().tolist())
    max_iter = trial.suggest_categorical("max_iter", df_tabu["max_iter"].unique().tolist())
    init_method = trial.suggest_categorical("init_method", df_tabu["init_method"].unique().tolist())

    # Filtrar apenas as execuções com esses parâmetros
    subset = df_tabu[(df_tabu["T_iter"] == T_iter) &
                (df_tabu["max_iter"] == max_iter) &
                (df_tabu["init_method"] == init_method)]

    alpha = 1  # Peso para conflitos
    beta = 0   # Peso para tempo
    score = alpha * subset["conflitos_norm"].mean() + beta * subset["tempo_norm"].mean()
    

    return score  # O Optuna busca minimizar esse valor


In [22]:
n_trials = df_tabu[["T_iter", "max_iter", "init_method"]].drop_duplicates().shape[0]

print(n_trials)

# Criar e rodar a otimização
study = optuna.create_study(direction="minimize")
study.optimize(objective_v4, n_trials=n_trials)  # Ajuste o número de trials conforme necessário

# Exibir os melhores parâmetros encontrados
print("Melhores parâmetros encontrados:")
print(study.best_params)

[I 2025-04-07 15:23:07,200] A new study created in memory with name: no-name-21bd01fc-3c4a-49be-9390-c768fab0ff96
[I 2025-04-07 15:23:07,211] Trial 0 finished with value: 0.24393939393939393 and parameters: {'T_iter': 75, 'max_iter': 10000, 'init_method': 'pseudogreedy_v3'}. Best is trial 0 with value: 0.24393939393939393.
[I 2025-04-07 15:23:07,215] Trial 1 finished with value: 0.24643939393939393 and parameters: {'T_iter': 25, 'max_iter': 10000, 'init_method': 'random'}. Best is trial 0 with value: 0.24393939393939393.
[I 2025-04-07 15:23:07,219] Trial 2 finished with value: 0.3259848484848485 and parameters: {'T_iter': 10, 'max_iter': 100, 'init_method': 'random'}. Best is trial 0 with value: 0.24393939393939393.
[I 2025-04-07 15:23:07,223] Trial 3 finished with value: 0.3712878787878787 and parameters: {'T_iter': 5, 'max_iter': 1000, 'init_method': 'pseudogreedy_v3'}. Best is trial 0 with value: 0.24393939393939393.
[I 2025-04-07 15:23:07,228] Trial 4 finished with value: 0.3156060

48


[I 2025-04-07 15:23:07,407] Trial 31 finished with value: 0.24393939393939393 and parameters: {'T_iter': 75, 'max_iter': 10000, 'init_method': 'pseudogreedy_v3'}. Best is trial 0 with value: 0.24393939393939393.
[I 2025-04-07 15:23:07,414] Trial 32 finished with value: 0.24393939393939393 and parameters: {'T_iter': 75, 'max_iter': 10000, 'init_method': 'pseudogreedy_v3'}. Best is trial 0 with value: 0.24393939393939393.
[I 2025-04-07 15:23:07,421] Trial 33 finished with value: 0.24393939393939393 and parameters: {'T_iter': 75, 'max_iter': 10000, 'init_method': 'pseudogreedy_v3'}. Best is trial 0 with value: 0.24393939393939393.
[I 2025-04-07 15:23:07,429] Trial 34 finished with value: 0.3156060606060606 and parameters: {'T_iter': 10, 'max_iter': 10000, 'init_method': 'random'}. Best is trial 0 with value: 0.24393939393939393.
[I 2025-04-07 15:23:07,437] Trial 35 finished with value: 0.2727272727272727 and parameters: {'T_iter': 75, 'max_iter': 100, 'init_method': 'pseudogreedy_v3'}. Be

Melhores parâmetros encontrados:
{'T_iter': 75, 'max_iter': 10000, 'init_method': 'pseudogreedy_v3'}


## SIMULATED ANNEALING

In [1]:
files = ['4-FullIns_4', 'ash331GPIA', 'DSJC250.1', 'le450_15a', 'qg.order30', 'queen13_13']
columns_sa = ['conflitos', 'tempo_us', 'T_max', 'T_min', 
                'max_iter', 'max_worsen', 'alpha', 'iter_found_best',
                'curr_Temp', 'filename', 'num_vertices', 'num_edges', 'init_method' ]

In [33]:
dfs = []

for file in files:
    df = pd.read_csv(f'results_sa_v2/analysis/{file}.txt', sep=';', names=columns_sa, header=None)
    df['conflitos_norm'] = (df["conflitos"] - df["conflitos"].min()) / (df["conflitos"].max() - df["conflitos"].min() if df["conflitos"].max()!= df["conflitos"].min() else 0)
    df['tempo_norm'] = (df["tempo_us"] - df["tempo_us"].min()) / (df["tempo_us"].max() - df["tempo_us"].min() if df["tempo_us"].max()!= df["tempo_us"].min() else 0)
    dfs.append(df)

df_sa = pd.concat(dfs, ignore_index=True)
df_sa.head()

,conflitos,tempo_us,T_max,T_min,max_iter,max_worsen,alpha,iter_found_best,curr_Temp,filename,num_vertices,num_edges,init_method,conflitos_norm,tempo_norm
0,636,850800,100,10,1000,1000,0.6,7354,7,instances/4-FullIns_4.col,690,6650,random,0.925743,0.002494
1,616,898268,100,10,1000,1000,0.6,8085,7,instances/4-FullIns_4.col,690,6650,random,0.892739,0.006902
2,612,895010,100,10,1000,1000,0.6,7669,7,instances/4-FullIns_4.col,690,6650,random,0.886139,0.006600
3,649,885161,100,10,1000,1000,0.6,5471,7,instances/4-FullIns_4.col,690,6650,random,0.947195,0.005685
4,635,841120,100,10,1000,1000,0.6,8415,7,instances/4-FullIns_4.col,690,6650,random,0.924092,0.001595


In [37]:
df_sa['conflitos_norm'] = (df_sa["conflitos"] - df_sa["conflitos"].min()) / (df_sa["conflitos"].max() - df_sa["conflitos"].min() if df_sa["conflitos"].max()!= df_sa["conflitos"].min() else 0)
df_sa['tempo_norm'] = (df_sa["tempo_us"] - df_sa["tempo_us"].min()) / (df_sa["tempo_us"].max() - df_sa["tempo_us"].min() if df_sa["tempo_us"].max()!= df_sa["tempo_us"].min() else 0)
df_sa[df_sa['init_method']=='random']
def objective_v5(trial):
    """Função objetivo para o Optuna, buscando a melhor configuração de parâmetros já testada."""

    # Optuna irá escolher um conjunto de parâmetros dentro dos valores existentes
    T_max = trial.suggest_categorical("T_max", df_sa["T_max"].unique().tolist())
    alpha = trial.suggest_categorical("alpha", df_sa["alpha"].unique().tolist())
    # init_method = trial.suggest_categorical("init_method", df_sa["init_method"].unique().tolist())

    # Filtrar apenas as execuções com esses parâmetros
    subset = df_sa[(df_sa["T_max"] == T_max) &
                (df_sa["alpha"] == alpha) ]

    alpha = 1  # Peso para conflitos
    beta = 0   # Peso para tempo
    score = alpha * subset["conflitos_norm"].mean() + beta * subset["tempo_norm"].mean()
    

    return score  # O Optuna busca minimizar esse valor


In [38]:
n_trials = df_sa[["T_max", "alpha", "filename"]].drop_duplicates().shape[0]

print(n_trials)

# Criar e rodar a otimização
study = optuna.create_study(direction="minimize")
study.optimize(objective_v5, n_trials=n_trials)  # Ajuste o número de trials conforme necessário

# Exibir os melhores parâmetros encontrados
print("Melhores parâmetros encontrados:")
print(study.best_params)

[I 2025-04-20 23:19:01,489] A new study created in memory with name: no-name-ac949949-400a-4d52-a720-dbc591a0bbea
[I 2025-04-20 23:19:01,492] Trial 0 finished with value: 0.3453586497890295 and parameters: {'T_max': 1000, 'alpha': 0.8}. Best is trial 0 with value: 0.3453586497890295.
[I 2025-04-20 23:19:01,493] Trial 1 finished with value: 0.35037505860290674 and parameters: {'T_max': 10000, 'alpha': 0.7}. Best is trial 0 with value: 0.3453586497890295.
[I 2025-04-20 23:19:01,496] Trial 2 finished with value: 0.3469936708860759 and parameters: {'T_max': 100, 'alpha': 0.7}. Best is trial 0 with value: 0.3453586497890295.
[I 2025-04-20 23:19:01,497] Trial 3 finished with value: 0.3472222222222222 and parameters: {'T_max': 5000, 'alpha': 0.8}. Best is trial 0 with value: 0.3453586497890295.
[I 2025-04-20 23:19:01,499] Trial 4 finished with value: 0.34172526957337085 and parameters: {'T_max': 10000, 'alpha': 0.9}. Best is trial 4 with value: 0.34172526957337085.
[I 2025-04-20 23:19:01,501]

96


[I 2025-04-20 23:19:01,692] Trial 83 finished with value: 0.33939287388654477 and parameters: {'T_max': 100, 'alpha': 0.9}. Best is trial 6 with value: 0.33939287388654477.
[I 2025-04-20 23:19:01,695] Trial 84 finished with value: 0.33939287388654477 and parameters: {'T_max': 100, 'alpha': 0.9}. Best is trial 6 with value: 0.33939287388654477.
[I 2025-04-20 23:19:01,698] Trial 85 finished with value: 0.34172526957337085 and parameters: {'T_max': 10000, 'alpha': 0.9}. Best is trial 6 with value: 0.33939287388654477.
[I 2025-04-20 23:19:01,700] Trial 86 finished with value: 0.33939287388654477 and parameters: {'T_max': 100, 'alpha': 0.9}. Best is trial 6 with value: 0.33939287388654477.
[I 2025-04-20 23:19:01,702] Trial 87 finished with value: 0.34478434130332863 and parameters: {'T_max': 100, 'alpha': 0.8}. Best is trial 6 with value: 0.33939287388654477.
[I 2025-04-20 23:19:01,705] Trial 88 finished with value: 0.33939287388654477 and parameters: {'T_max': 100, 'alpha': 0.9}. Best is t

Melhores parâmetros encontrados:
{'T_max': 100, 'alpha': 0.9}
